In [61]:
import numpy as np
import pandas as pd
from scipy import stats
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [62]:
vars_df = pd.read_pickle('../../multi-age-mapper/Analysis Female - Variance Study/female_vars_testB_convergence9.pkl')
    
with open('../../agemapper/Analsysis Female New/varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
with open('../../agemapper/Analsysis Female New/vars_categories.txt') as f:
    vars_categories = f.readlines()
    vars_categories = [l.strip('\n\r') for l in vars_categories]
    vars_categories = np.array(vars_categories)
with open('../../agemapper/Analsysis Female New/varsVARS.txt') as f:
    varsVARS = f.readlines()
    varsVARS = [l.strip('\n\r') for l in varsVARS]
    varsVARS = np.array(varsVARS)

In [63]:
vID = []
for idx in range(len(vars_df.iloc[0].dataframe)):
    vID.append(varsVARS[np.where(varsHeader == vars_df.iloc[0].dataframe.iloc[idx].names)[0][0]])
# df['Variable ID'] = vID

for idx in range(len(vars_df)):
    vars_df.iloc[idx].dataframe['Variable ID'] = vID

In [64]:
current_mods = vars_df.modality.values
modality_names = [
    'T1 Nonlinear', 'T2 Nonlinear', 'FA', 'MD', 'rsfMRI-0', 'SWI', 'tfMRI-1', 'Summed Tracts', 'MO',
    'TBSS ISOVF', 'L2', 'rsfMRI-2', 'rsfMRI-10', 'rsfMRI-5', 'rsfMRI-21', 'T2 Lessions', 'TBSS FA', 'TBSS L3'
]
for idx, mod in enumerate(current_mods):
    vars_df.loc[vars_df["modality"] == mod, "modality"] = modality_names[idx]

In [65]:
vars_df

,modality,bonf,fdr,dataframe
0,T1 Nonlinear,5.442009,3.722608,idx ...
1,T2 Nonlinear,5.442009,3.643544,idx ...
2,FA,5.442009,3.258847,idx ...
3,MD,5.442009,3.452753,idx ...
4,rsfMRI-0,5.442009,3.389271,idx ...
5,SWI,5.442009,3.490886,idx ...
6,tfMRI-1,5.442009,3.608196,idx ...
7,Summed Tracts,5.442009,3.403591,idx ...
8,MO,5.442009,3.494445,idx ...
9,TBSS ISOVF,5.442009,3.308815,idx ...


In [66]:

    

vars_df = vars_df[(vars_df.modality == 'T1 Nonlinear') |
                (vars_df.modality == 'T2 Nonlinear') |
                (vars_df.modality == 'FA') |
                  (vars_df.modality == 'MD') |
                (vars_df.modality == 'rsfMRI-0') |
                (vars_df.modality == 'SWI') |
                  (vars_df.modality == 'tfMRI-1') |
                (vars_df.modality == 'Summed Tracts')   |
                (vars_df.modality == 'MO') |
                  (vars_df.modality == 'TBSS ISOVF') |
                (vars_df.modality == 'L2') |
                (vars_df.modality == 'rsfMRO-2') |
                  (vars_df.modality == 'rsfMRO-10') |
                (vars_df.modality == 'rsfMRI-5') |
                (vars_df.modality == 'rsfMRI-21') |
                  (vars_df.modality == 'T2 Lessions') |
                (vars_df.modality == 'TBSS FA') |
                (vars_df.modality == 'TBSS L3') 
                  
]


vars_over_bThr_list = []

for idx in range(len(vars_df)):
    df = vars_df.iloc[idx].dataframe
    bThr = vars_df.iloc[idx].bonf
    vars_over_bThr = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False).names.to_list()
    for var in vars_over_bThr:
        vars_over_bThr_list.append(var)
        
print("Number of unique vars across all modalities: ", len(set(vars_over_bThr_list)))

unique_vars_over_bTHr = list(set(vars_over_bThr_list))
print(len(unique_vars_over_bTHr))

unique_vars_over_bThr_categories = []

for var in unique_vars_over_bTHr:
    idx = list(varsHeader).index(var)
    category = vars_categories[idx]
    unique_vars_over_bThr_categories.append(category)
    
modalities = []
pearson_rs_list = []
logPs = []
var_ids = []
for var in unique_vars_over_bTHr:
    modalities_with_var = []
    pearsonRs_with_var = []
    logPs_with_var = []
    varIDs_with_var = []
    for idx in range(len(vars_df)):
        df = vars_df.iloc[idx].dataframe
        modality = vars_df.iloc[idx].modality
        bThr = vars_df.iloc[idx].bonf
        df = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False)
        vars_over_bThr = df.names.to_list()
        pearson_r_over_bThr = df.pearson_r.to_list()
        logps_over_bTHr = df.log_p_values.to_list()
        var_ids_over_bTHr = df['Variable ID'].to_list()
        if var in vars_over_bThr:
            modalities_with_var.append(modality)
            pearsonRs_with_var.append(pearson_r_over_bThr[vars_over_bThr.index(var)])
            logPs_with_var.append(logps_over_bTHr[vars_over_bThr.index(var)])
            varIDs_with_var.append(var_ids_over_bTHr[vars_over_bThr.index(var)])
    modalities.append(modalities_with_var)
    pearson_rs_list.append(pearsonRs_with_var)
    logPs.append(logPs_with_var)
    var_ids.append(varIDs_with_var)
    
df_vars_over_bThr = pd.DataFrame.from_dict({
    'names': unique_vars_over_bTHr,
    'Categories': unique_vars_over_bThr_categories,
    'modalities': modalities,
    'pearson_r': pearson_rs_list,
    'log_p_values': logPs,
    'var_ids': var_ids
})

for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    
    for name in sorted(names):
        
#         print(df_vars_over_bThr[df_vars_over_bThr.names == name].var_ids.to_list()[0][0])
        
        print('Var: {}, ID: {}'.format(name, 
                df_vars_over_bThr[df_vars_over_bThr.names == name].var_ids.to_list()[0][0]))
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        logps = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].log_p_values.to_list()
        print('   Modalities:')
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        _, logps = zip(*sorted(zip(prs_abs[0], logps[0]), reverse=True))
        
        for i, mod in enumerate(mods):
            print('   -> {} ({:.3f}) [{:.3f}]'.format(mod, prs[i], logps[i]))
        
    print('\n')

Number of unique vars across all modalities:  117
117
Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (0.0), ID: 1558-0.0
   Modalities:
   -> TBSS ISOVF (0.100) [6.286]
   -> FA (0.094) [5.584]
Var: Amount of alcohol drunk on a typical drinking day (0.0), ID: 20403-0.0
   Modalities:
   -> L2 (0.124) [6.312]
   -> MD (0.119) [5.824]
Var: Frequency of consuming six or more units of alcohol (0.0), ID: 20416-0.0
   Modalities:
   -> L2 (0.131) [6.932]
   -> MD (0.125) [6.373]
Var: Frequency of drinking alcohol (0.0), ID: 20414-0.0
   Modalities:
   -> TBSS ISOVF (0.132) [7.629]
   -> MD (0.127) [7.148]
   -> L2 (0.122) [6.622]
   -> rsfMRI-0 (0.122) [6.591]
   -> FA (0.119) [6.350]
   -> TBSS L3 (0.118) [6.254]
   -> TBSS FA (0.116) [6.051]
   -> MO (0.116) [6.009]


Category: Blood Assays
------------------------------------------------
Var: C-reactive protein (0.0), ID: 30710-0.0
   Modalities:
   -> rsfMRI-5 (-0.115) [7.623]
   -> rsfM

   Modalities:
   -> rsfMRI-0 (-0.231) [5.749]
Var: Speed of sound through heel (left) (2.0), ID: 4103-2.0
   Modalities:
   -> rsfMRI-0 (-0.219) [21.571]
   -> rsfMRI-21 (-0.199) [17.793]
   -> rsfMRI-5 (-0.175) [13.961]
   -> tfMRI-1 (-0.151) [10.569]
   -> MO (-0.146) [9.885]
   -> MD (-0.135) [8.465]
   -> L2 (-0.134) [8.384]
   -> TBSS ISOVF (-0.132) [8.205]
   -> SWI (-0.125) [7.395]
   -> FA (-0.123) [7.199]
   -> Summed Tracts (-0.121) [6.996]
   -> TBSS L3 (-0.119) [6.735]
   -> T1 Nonlinear (-0.107) [5.559]
Var: Speed of sound through heel (right) (1.0), ID: 4122-1.0
   Modalities:
   -> rsfMRI-0 (-0.262) [7.329]
Var: Speed of sound through heel (right) (2.0), ID: 4122-2.0
   Modalities:
   -> rsfMRI-0 (-0.227) [23.183]
   -> rsfMRI-21 (-0.209) [19.746]
   -> rsfMRI-5 (-0.183) [15.215]
   -> tfMRI-1 (-0.156) [11.233]
   -> MO (-0.143) [9.547]
   -> Summed Tracts (-0.125) [7.419]
   -> SWI (-0.125) [7.384]
   -> TBSS ISOVF (-0.125) [7.372]
   -> MD (-0.120) [6.876]
   -> TBSS 

In [67]:
df_vars_over_bThr

,names,Categories,modalities,pearson_r,log_p_values,var_ids
0,Trunk fat percentage (2.0),Physical Measurements,"[FA, TBSS ISOVF, rsfMRI-21]","[-0.09715687350746668, -0.11987172807075169, -...","[5.763564192869423, 8.46341741291554, 6.840140...","[23127-2.0, 23127-2.0, 23127-2.0]"
1,Arm BMD (bone mineral density) (right) (2.0),Skeletal Measurements,"[T1 Nonlinear, T2 Nonlinear, FA, MD, rsfMRI-0,...","[-0.1759843028723604, -0.17041418541477701, -0...","[6.467141707672262, 6.096909011577523, 6.77603...","[23223-2.0, 23223-2.0, 23223-2.0, 23223-2.0, 2..."
2,Femur neck BMD (bone mineral density) (left) (...,Skeletal Measurements,"[T1 Nonlinear, FA, MD, rsfMRI-0, SWI, tfMRI-1,...","[-0.12932102841911214, -0.13333025071305948, -...","[9.069060758204804, 9.601038418969425, 7.37163...","[23299-2.0, 23299-2.0, 23299-2.0, 23299-2.0, 2..."
3,Body fat percentage (0.0),Physical Measurements,[TBSS ISOVF],[-0.09445970145981941],[5.606727433653842],[23099-0.0]
4,Arm fat percentage (right) (2.0),Physical Measurements,[TBSS ISOVF],[-0.10721120229636819],[6.89149936382775],[23119-2.0]
...,...,...,...,...,...,...
112,Body surface area (2.0),Physical Measurements,"[FA, MO, TBSS ISOVF, rsfMRI-21]","[-0.10753668912741736, -0.10303788916050441, -...","[6.582318711400024, 6.090721253697018, 6.63396...","[22427-2.0, 22427-2.0, 22427-2.0, 22427-2.0]"
113,Femur wards BMD (bone mineral density) (right)...,Skeletal Measurements,"[T1 Nonlinear, T2 Nonlinear, FA, MD, rsfMRI-0,...","[-0.14033366984244078, -0.10158406915188357, -...","[10.286487203402034, 5.673718299479887, 12.138...","[23218-2.0, 23218-2.0, 23218-2.0, 23218-2.0, 2..."
114,Ribs BMD (bone mineral density) (2.0),Skeletal Measurements,"[T1 Nonlinear, T2 Nonlinear, FA, MD, rsfMRI-0,...","[-0.13375230135028893, -0.10932134929017973, -...","[9.661987566890254, 6.655956091127037, 13.8037...","[23233-2.0, 23233-2.0, 23233-2.0, 23233-2.0, 2..."
115,Arms tissue fat percentage (2.0),Physical Measurements,[TBSS ISOVF],[-0.09867300047748859],[5.52884997385671],[23259-2.0]


In [68]:
vars_df_converged = pd.read_pickle('female_vars_testB_convergence9.pkl')

current_mods = vars_df_converged.modality.values
modality_names = [
    'T1 Nonlinear', 'T2 Nonlinear', 'FA', 'MD', 'rsfMRI-0', 'SWI', 'tfMRI-1', 'Summed Tracts', 'MO',
    'TBSS ISOVF', 'L2', 'rsfMRI-2', 'rsfMRI-10', 'rsfMRI-5', 'rsfMRI-21', 'T2 Lessions', 'TBSS FA', 'TBSS L3'
]
for idx, mod in enumerate(current_mods):
    vars_df_converged.loc[vars_df_converged["modality"] == mod, "modality"] = modality_names[idx]

current_mods = vars_df_converged.modality.values

# modality_names = [
#     'T1_nonlinear_swin_c',           #1
#     'T2_nonlinear_swin_c',           #2
#     'tbss_FA_swin_c',                #3
#     'tbss_MD_swin_c',                #4
#     'rsfmri_0_swin_c',               #5
#     'swi_swin_c',                    #6
#     'tfmri_swin_1_c',                #7
#     'tracts_swin_c',                 #8
#     'tbss_MO_swin_c',                #9
#     'tbss_ISOVF_swin_s_c',           #10
#     'tbss_L2_swin_c',                #11
#     'rsfmri_2_swin_c',               #12
#     'rsfmri_10_swin_c',              #13
#     'rsfmri_5_swin_c',               #14
#     'rsfmri_21_swin_c',              #15
#     'T2_lessions_swin_c',            #16
#     'tbss_FA_s_swin_c',              #17
#     'tbss_L3_s_swin_c',              #18
# ]


vID = []
for idx in range(len(vars_df_converged.iloc[0].dataframe)):
    vID.append(varsVARS[np.where(varsHeader == vars_df_converged.iloc[0].dataframe.iloc[idx].names)[0][0]])
# df['Variable ID'] = vID

for idx in range(len(vars_df_converged)):
    vars_df_converged.iloc[idx].dataframe['Variable ID'] = vID




# for idx, mod in enumerate(current_mods):
#     vars_df_converged.loc[vars_df_converged["modality"] == mod, "modality"] = modality_names[idx]

vars_over_bThr_list_converged = []

for idx in range(len(vars_df_converged)):
    df = vars_df_converged.iloc[idx].dataframe
    bThr = vars_df_converged.iloc[idx].bonf
    vars_over_bThr = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False).names.to_list()
    for var in vars_over_bThr:
        vars_over_bThr_list_converged.append(var)
        
print("Number of unique vars across all modalities: ", len(set(vars_over_bThr_list_converged)))

unique_vars_over_bTHr_converged = list(set(vars_over_bThr_list_converged))
print(len(unique_vars_over_bTHr_converged))

unique_vars_over_bThr_categories_converged = []

for var in unique_vars_over_bTHr_converged:
    idx = list(varsHeader).index(var)
    category = vars_categories[idx]
    unique_vars_over_bThr_categories_converged.append(category)
    
modalities_converged = []
pearson_rs_list_converged = []
logPs_converged = []
var_ids = []
for var in unique_vars_over_bTHr_converged:
    modalities_with_var = []
    pearsonRs_with_var = []
    logPs_with_var = []
    varIDs_with_var = []
    for idx in range(len(vars_df_converged)):
        df = vars_df_converged.iloc[idx].dataframe
        modality = vars_df_converged.iloc[idx].modality
        bThr = vars_df_converged.iloc[idx].bonf
        df = df[df.log_p_values>=bThr].sort_values('log_p_values', ascending=False)
        vars_over_bThr = df.names.to_list()
        pearson_r_over_bThr = df.pearson_r.to_list()
        logps_over_bTHr = df.log_p_values.to_list()
        var_ids_over_bTHr = df['Variable ID'].to_list()
        if var in vars_over_bThr:
            modalities_with_var.append(modality)
            pearsonRs_with_var.append(pearson_r_over_bThr[vars_over_bThr.index(var)])
            logPs_with_var.append(logps_over_bTHr[vars_over_bThr.index(var)])
            varIDs_with_var.append(var_ids_over_bTHr[vars_over_bThr.index(var)])
    modalities_converged.append(modalities_with_var)
    pearson_rs_list_converged.append(pearsonRs_with_var)
    logPs_converged.append(logPs_with_var)
    var_ids.append(varIDs_with_var)
    
df_vars_over_bThr_converged = pd.DataFrame.from_dict({
    'names': unique_vars_over_bTHr_converged,
    'Categories': unique_vars_over_bThr_categories_converged,
    'modalities': modalities_converged,
    'pearson_r': pearson_rs_list_converged,
    'log_p_values': logPs_converged,
    'var_ids': var_ids
})

for cat in sorted(list(set(df_vars_over_bThr_converged.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr_converged[df_vars_over_bThr_converged.Categories == cat].names.to_list()
    for name in sorted(names):
        IDx = df_vars_over_bThr_converged[df_vars_over_bThr_converged.names == name].var_ids.to_list()[0][0]
        print('Var: {}, ID: {}'.format(name, IDx))
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        mods = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].modalities.to_list()
        prs = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].pearson_r.to_list()
        logps = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].log_p_values.to_list()
        print('   Modalities:')
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        _, logps = zip(*sorted(zip(prs_abs[0], logps[0]), reverse=True))
        
        for i, mod in enumerate(mods):
            print('   -> {} ({:.3f}) [{:.3f}]'.format(mod, prs[i], logps[i]))
        
    print('\n')

Number of unique vars across all modalities:  111
111
Category: Alcohol
------------------------------------------------
Var: Frequency of consuming six or more units of alcohol (0.0), ID: 20416-0.0
   Modalities:
   -> FA (0.121) [6.000]
Var: Frequency of drinking alcohol (0.0), ID: 20414-0.0
   Modalities:
   -> MD (0.127) [7.130]
   -> L2 (0.121) [6.518]
   -> FA (0.117) [6.137]
   -> TBSS FA (0.112) [5.660]
   -> TBSS ISOVF (0.110) [5.536]


Category: Blood Assays
------------------------------------------------
Var: C-reactive protein (0.0), ID: 30710-0.0
   Modalities:
   -> rsfMRI-5 (-0.096) [5.446]


Category: Cardiac & Circulartory Measurements
------------------------------------------------
Var: Systolic blood pressure, automated reading (0.0), ID: 4080-0.0
   Modalities:
   -> T2 Lessions (0.102) [6.081]
   -> TBSS FA (0.101) [6.024]
Var: Systolic blood pressure, automated reading (0.1), ID: 4080-0.1
   Modalities:
   -> T2 Lessions (0.100) [5.787]


Category: Diet
--------

   Modalities:
   -> Summed Tracts (-0.160) [5.644]
   -> rsfMRI-2 (-0.157) [5.496]
Var: Heel broadband ultrasound attenuation (left) (2.0), ID: 4101-2.0
   Modalities:
   -> rsfMRI-0 (-0.175) [13.927]
   -> rsfMRI-2 (-0.156) [11.117]
   -> tfMRI-1 (-0.148) [10.167]
   -> Summed Tracts (-0.143) [9.463]
   -> TBSS ISOVF (-0.130) [7.890]
   -> MO (-0.128) [7.767]
   -> rsfMRI-5 (-0.125) [7.343]
   -> L2 (-0.115) [6.348]
   -> TBSS L3 (-0.114) [6.217]
   -> T1 Nonlinear (-0.113) [6.187]
   -> rsfMRI-21 (-0.113) [6.131]
   -> FA (-0.112) [6.091]
   -> MD (-0.111) [5.956]
   -> TBSS FA (-0.107) [5.569]
Var: Heel broadband ultrasound attenuation (right) (0.0), ID: 4120-0.0
   Modalities:
   -> rsfMRI-2 (-0.171) [6.396]
   -> Summed Tracts (-0.158) [5.509]
Var: Heel broadband ultrasound attenuation (right) (1.0), ID: 4120-1.0
   Modalities:
   -> rsfMRI-0 (-0.240) [6.206]
Var: Heel broadband ultrasound attenuation (right) (2.0), ID: 4120-2.0
   Modalities:
   -> rsfMRI-0 (-0.180) [14.728]
   

In [69]:
new_mods = 0

Dcontrasts = []
Dlogps = []
Dcorrs = []
DvarCat = []
DvarDesc = []
DvarID = []

print('BONFERRONI === ', vars_df.iloc[0].bonf)
print('\n')

for cat in sorted(list(set(df_vars_over_bThr.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr[df_vars_over_bThr.Categories == cat].names.to_list()
    for name in sorted(names):
        print('Var: {}'.format(name))
        mods = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        logps = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].log_p_values.to_list()
        idxs = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].var_ids.to_list()
        
        mods0 = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].modalities.to_list()
        prs0 = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].pearson_r.to_list()
        logps0 = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].log_p_values.to_list()
        idxs0 = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].var_ids.to_list()
        
        print('   Modalities:')
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        _, logps = zip(*sorted(zip(prs_abs[0], logps[0]), reverse=True))
        _, idxs = zip(*sorted(zip(prs_abs[0], idxs[0]), reverse=True))
        
        for i, mod in enumerate(mods):
            print('   -> {} >> {} << ({:.3f}) [{:.3f}]'.format(mod, idxs[0], prs[i], logps[i]))
        
        
        if len(mods0) == 0:
            print('======== ATTENTION! NEW VAR ! ==========')
            new_mods += 1
#         else:
#             prs_abs0 = np.abs(prs0).tolist()
#             _, mods0 = zip(*sorted(zip(prs_abs0[0], mods0[0]), reverse=True))
#             _, prs0 = zip(*sorted(zip(prs_abs0[0], prs0[0]), reverse=True))
        
#             print('   -> {} ({:.3f})'.format(mods0[0], prs0[0]))

            Dcontrasts.append(mod)
            Dlogps.append(logps[i])
            Dcorrs.append(prs[i])
            DvarCat.append(cat)
            DvarDesc.append(name)
            DvarID.append(idxs[0])
        
    print('\n')
    
print('New Mods Missed by SWIN: ', new_mods)

BONFERRONI ===  5.442009159140952


Category: Alcohol
------------------------------------------------
Var: Alcohol intake frequency. (0.0)
   Modalities:
   -> TBSS ISOVF >> 1558-0.0 << (0.100) [6.286]
   -> FA >> 1558-0.0 << (0.094) [5.584]
======== ATTENTION! NEW VAR ! ==========
Var: Amount of alcohol drunk on a typical drinking day (0.0)
   Modalities:
   -> L2 >> 20403-0.0 << (0.124) [6.312]
   -> MD >> 20403-0.0 << (0.119) [5.824]
======== ATTENTION! NEW VAR ! ==========
Var: Frequency of consuming six or more units of alcohol (0.0)
   Modalities:
   -> L2 >> 20416-0.0 << (0.131) [6.932]
   -> MD >> 20416-0.0 << (0.125) [6.373]
Var: Frequency of drinking alcohol (0.0)
   Modalities:
   -> TBSS ISOVF >> 20414-0.0 << (0.132) [7.629]
   -> MD >> 20414-0.0 << (0.127) [7.148]
   -> L2 >> 20414-0.0 << (0.122) [6.622]
   -> rsfMRI-0 >> 20414-0.0 << (0.122) [6.591]
   -> FA >> 20414-0.0 << (0.119) [6.350]
   -> TBSS L3 >> 20414-0.0 << (0.118) [6.254]
   -> TBSS FA >> 20414-0.0 << (0.116

   Modalities:
   -> TBSS ISOVF >> 23112-2.0 << (-0.129) [9.744]
   -> rsfMRI-21 >> 23112-2.0 << (-0.117) [8.085]
   -> FA >> 23112-2.0 << (-0.104) [6.559]
   -> rsfMRI-5 >> 23112-2.0 << (-0.103) [6.423]
   -> Summed Tracts >> 23112-2.0 << (-0.097) [5.730]
Var: Leg fat percentage (left) (2.0)
   Modalities:
   -> TBSS ISOVF >> 23115-2.0 << (-0.096) [5.670]
======== ATTENTION! NEW VAR ! ==========
Var: Leg fat percentage (right) (2.0)
   Modalities:
   -> TBSS ISOVF >> 23111-2.0 << (-0.097) [5.709]
======== ATTENTION! NEW VAR ! ==========
Var: Leg fat-free mass (right) (2.0)
   Modalities:
   -> rsfMRI-21 >> 23113-2.0 << (-0.095) [5.526]
======== ATTENTION! NEW VAR ! ==========
Var: Legs total mass (2.0)
   Modalities:
   -> FA >> 23277-2.0 << (-0.115) [7.262]
   -> TBSS ISOVF >> 23277-2.0 << (-0.114) [7.198]
   -> rsfMRI-5 >> 23277-2.0 << (-0.114) [7.183]
   -> tfMRI-1 >> 23277-2.0 << (-0.099) [5.599]
======== ATTENTION! NEW VAR ! ==========
Var: Total mass (2.0)
   Modalities:
   -> T

   Modalities:
   -> rsfMRI-21 >> 23216-2.0 << (-0.268) [8.284]
Var: Femur upper neck BMD (bone mineral density) T-score (left) (2.0)
   Modalities:
   -> rsfMRI-21 >> 23296-2.0 << (-0.249) [7.635]
Var: Femur upper neck BMD (bone mineral density) T-score (right) (2.0)
   Modalities:
   -> rsfMRI-21 >> 23217-2.0 << (-0.266) [8.158]
======== ATTENTION! NEW VAR ! ==========
Var: Femur wards BMD (bone mineral density) (left) (2.0)
   Modalities:
   -> rsfMRI-5 >> 23297-2.0 << (-0.263) [36.031]
   -> rsfMRI-21 >> 23297-2.0 << (-0.263) [35.803]
   -> rsfMRI-0 >> 23297-2.0 << (-0.210) [23.030]
   -> tfMRI-1 >> 23297-2.0 << (-0.198) [20.411]
   -> SWI >> 23297-2.0 << (-0.194) [19.724]
   -> MO >> 23297-2.0 << (-0.169) [15.097]
   -> TBSS ISOVF >> 23297-2.0 << (-0.160) [13.532]
   -> FA >> 23297-2.0 << (-0.159) [13.447]
   -> T1 Nonlinear >> 23297-2.0 << (-0.155) [12.756]
   -> MD >> 23297-2.0 << (-0.146) [11.415]
   -> L2 >> 23297-2.0 << (-0.142) [10.873]
   -> TBSS L3 >> 23297-2.0 << (-0.136)

   Modalities:
   -> rsfMRI-21 >> 23236-2.0 << (-0.351) [64.906]
   -> rsfMRI-5 >> 23236-2.0 << (-0.341) [61.318]
   -> rsfMRI-0 >> 23236-2.0 << (-0.308) [49.544]
   -> tfMRI-1 >> 23236-2.0 << (-0.231) [27.676]
   -> SWI >> 23236-2.0 << (-0.225) [26.370]
   -> MO >> 23236-2.0 << (-0.215) [24.061]
   -> FA >> 23236-2.0 << (-0.196) [19.969]
   -> TBSS ISOVF >> 23236-2.0 << (-0.195) [19.775]
   -> L2 >> 23236-2.0 << (-0.176) [16.216]
   -> MD >> 23236-2.0 << (-0.175) [16.192]
   -> TBSS L3 >> 23236-2.0 << (-0.168) [14.947]
   -> T1 Nonlinear >> 23236-2.0 << (-0.158) [13.163]
   -> Summed Tracts >> 23236-2.0 << (-0.157) [13.130]
   -> TBSS FA >> 23236-2.0 << (-0.144) [11.138]
   -> T2 Nonlinear >> 23236-2.0 << (-0.124) [8.355]
Var: Total BMD (bone mineral density) T-score (2.0)
   Modalities:
   -> rsfMRI-21 >> 23239-2.0 << (-0.353) [65.888]
   -> rsfMRI-5 >> 23239-2.0 << (-0.345) [62.696]
   -> rsfMRI-0 >> 23239-2.0 << (-0.309) [50.089]
   -> tfMRI-1 >> 23239-2.0 << (-0.232) [28.072]
   -

In [70]:
dfT = pd.DataFrame.from_dict({
    'Contrast': Dcontrasts,
    '-log(p)':Dlogps, 
    'Correlation (Pearson r)': Dcorrs,
    'Variable Category': DvarCat,
    'Variable Description': DvarDesc,
    'Variable ID': DvarID
    
})

categories = sorted(list(set(dfT['Variable Category'].to_list())))

first_df = True

for cat in categories:
    df_tmp = dfT[(dfT['Variable Category']==cat)].sort_values('-log(p)', ascending=False)
#     df_tmp = df_tmp.head(10)
    if len(df_tmp) == 0:
        continue
    if first_df == True:
        df_print = df_tmp
        first_df = False
    else:
        df_print = pd.concat([df_print, df_tmp])
        


df_print = df_print.round(decimals=3)


df_print.to_csv('nIDPs_captured_by_CNN.csv', index=False)

df_print

,Contrast,-log(p),Correlation (Pearson r),Variable Category,Variable Description,Variable ID
1,MD,5.824,0.119,Alcohol,Amount of alcohol drunk on a typical drinking ...,20403-0.0
0,FA,5.584,0.094,Alcohol,Alcohol intake frequency. (0.0),1558-0.0
2,T1 Nonlinear,6.237,0.100,Medical History,"Non-cancer illness code, self-reported (1134 -...",20002-0.1134
4,rsfMRI-21,5.619,0.094,Medical History,Operative procedures - main OPCS4 (X369 - X36....,41200-0.9159
5,L2,5.597,0.094,Medical History,Treatment/medication code (1140883066 - insuli...,20003-0.1140883066
3,T1 Nonlinear,5.454,0.092,Medical History,Operative procedures - OPCS4 (A842 - A84.2 Ele...,41272-0.532
21,FA,7.082,-0.113,Physical Measurements,Total mass (2.0),23283-2.0
10,TBSS ISOVF,6.891,-0.107,Physical Measurements,Arm fat percentage (right) (2.0),23119-2.0
7,rsfMRI-21,6.881,-0.111,Physical Measurements,Android total mass (2.0),23248-2.0
8,FA,6.289,-0.102,Physical Measurements,Arm fat mass (right) (2.0),23120-2.0


In [72]:
new_mods = 0


Dcontrasts = []
Dlogps = []
Dcorrs = []
DvarCat = []
DvarDesc = []
DvarID = []


print('BONFERRONI === ', vars_df_converged.iloc[0].bonf)
print('\n')

for cat in sorted(list(set(df_vars_over_bThr_converged.Categories.to_list()))):
    print("Category: {}".format(cat))
    print('------------------------------------------------')
    names = df_vars_over_bThr_converged[df_vars_over_bThr_converged.Categories == cat].names.to_list()
    for name in sorted(names):
        print('Var: {}'.format(name))
        mods = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].modalities.to_list()
        prs = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].pearson_r.to_list()
        logps = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].log_p_values.to_list()
        idxs = df_vars_over_bThr_converged[(df_vars_over_bThr_converged.Categories == cat) & (df_vars_over_bThr_converged.names == name)].var_ids.to_list()

        mods0 = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].modalities.to_list()
        prs0 = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].pearson_r.to_list()
        logps0 = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].log_p_values.to_list()
        idxs0 = df_vars_over_bThr[(df_vars_over_bThr.Categories == cat) & (df_vars_over_bThr.names == name)].var_ids.to_list()
        
        
        print('   Modalities:')
        
        prs_abs = np.abs(prs).tolist()
        _, mods = zip(*sorted(zip(prs_abs[0], mods[0]), reverse=True))
        _, prs = zip(*sorted(zip(prs_abs[0], prs[0]), reverse=True))
        _, logps = zip(*sorted(zip(prs_abs[0], logps[0]), reverse=True))
        _, idxs = zip(*sorted(zip(prs_abs[0], idxs[0]), reverse=True))
        
        for i, mod in enumerate(mods):
            print('   -> {} >> {} << ({:.3f}) [{:.3f}]'.format(mod, idxs[0], prs[i], logps[i]))
        
        if len(mods0) == 0:
            print('======== ATTENTION! NEW VAR ! ==========')
            new_mods += 1
            
            Dcontrasts.append(mod)
            Dlogps.append(logps[i])
            Dcorrs.append(prs[i])
            DvarCat.append(cat)
            DvarDesc.append(name)
            DvarID.append(idxs[0])

            
#         else:
#             prs_abs0 = np.abs(prs0).tolist()
#             _, mods0 = zip(*sorted(zip(prs_abs0[0], mods0[0]), reverse=True))
#             _, prs0 = zip(*sorted(zip(prs_abs0[0], prs0[0]), reverse=True))
        
#             print('   -> {} ({:.3f})'.format(mods0[0], prs0[0]))
        
    print('\n')
    
print('New Mods Missed by CNN: ', new_mods)

BONFERRONI ===  5.442009159140952


Category: Alcohol
------------------------------------------------
Var: Frequency of consuming six or more units of alcohol (0.0)
   Modalities:
   -> FA >> 20416-0.0 << (0.121) [6.000]
Var: Frequency of drinking alcohol (0.0)
   Modalities:
   -> MD >> 20414-0.0 << (0.127) [7.130]
   -> L2 >> 20414-0.0 << (0.121) [6.518]
   -> FA >> 20414-0.0 << (0.117) [6.137]
   -> TBSS FA >> 20414-0.0 << (0.112) [5.660]
   -> TBSS ISOVF >> 20414-0.0 << (0.110) [5.536]


Category: Blood Assays
------------------------------------------------
Var: C-reactive protein (0.0)
   Modalities:
   -> rsfMRI-5 >> 30710-0.0 << (-0.096) [5.446]


Category: Cardiac & Circulartory Measurements
------------------------------------------------
Var: Systolic blood pressure, automated reading (0.0)
   Modalities:
   -> T2 Lessions >> 4080-0.0 << (0.102) [6.081]
   -> TBSS FA >> 4080-0.0 << (0.101) [6.024]
Var: Systolic blood pressure, automated reading (0.1)
   Modalities:
   -> T2

   Modalities:
   -> rsfMRI-2 >> 23206-2.0 << (-0.251) [7.288]
   -> tfMRI-1 >> 23206-2.0 << (-0.225) [5.944]
   -> rsfMRI-21 >> 23206-2.0 << (-0.215) [5.476]
Var: Femur neck BMD (bone mineral density) (left) (2.0)
   Modalities:
   -> rsfMRI-2 >> 23299-2.0 << (-0.237) [29.197]
   -> rsfMRI-5 >> 23299-2.0 << (-0.190) [18.843]
   -> tfMRI-1 >> 23299-2.0 << (-0.176) [16.355]
   -> SWI >> 23299-2.0 << (-0.161) [13.649]
   -> rsfMRI-10 >> 23299-2.0 << (-0.146) [11.442]
   -> rsfMRI-21 >> 23299-2.0 << (-0.143) [10.957]
   -> Summed Tracts >> 23299-2.0 << (-0.138) [10.185]
   -> rsfMRI-0 >> 23299-2.0 << (-0.133) [9.611]
   -> MO >> 23299-2.0 << (-0.129) [8.998]
   -> T1 Nonlinear >> 23299-2.0 << (-0.123) [8.279]
   -> T2 Nonlinear >> 23299-2.0 << (-0.115) [7.261]
   -> MD >> 23299-2.0 << (-0.108) [6.494]
   -> TBSS ISOVF >> 23299-2.0 << (-0.101) [5.722]
Var: Femur neck BMD (bone mineral density) (right) (2.0)
   Modalities:
   -> rsfMRI-2 >> 23208-2.0 << (-0.237) [28.473]
   -> tfMRI-1 >> 23

   Modalities:
   -> rsfMRI-2 >> 23233-2.0 << (-0.227) [26.835]
   -> rsfMRI-5 >> 23233-2.0 << (-0.202) [21.360]
   -> SWI >> 23233-2.0 << (-0.169) [15.135]
   -> rsfMRI-0 >> 23233-2.0 << (-0.166) [14.562]
   -> tfMRI-1 >> 23233-2.0 << (-0.164) [14.283]
   -> rsfMRI-21 >> 23233-2.0 << (-0.158) [13.199]
   -> rsfMRI-10 >> 23233-2.0 << (-0.157) [13.144]
   -> Summed Tracts >> 23233-2.0 << (-0.155) [12.743]
   -> MO >> 23233-2.0 << (-0.145) [11.285]
   -> MD >> 23233-2.0 << (-0.134) [9.746]
   -> FA >> 23233-2.0 << (-0.131) [9.306]
   -> L2 >> 23233-2.0 << (-0.128) [8.935]
   -> T1 Nonlinear >> 23233-2.0 << (-0.115) [7.307]
   -> T2 Nonlinear >> 23233-2.0 << (-0.114) [7.193]
   -> TBSS FA >> 23233-2.0 << (-0.100) [5.676]
   -> TBSS ISOVF >> 23233-2.0 << (-0.100) [5.634]
Var: Speed of sound through heel (0.0)
   Modalities:
   -> rsfMRI-0 >> 3146-0.0 << (-0.139) [6.364]
   -> rsfMRI-2 >> 3146-0.0 << (-0.130) [5.634]
Var: Speed of sound through heel (left) (2.0)
   Modalities:
   -> rsfMRI-

In [73]:
dfT = pd.DataFrame.from_dict({
    'Contrast': Dcontrasts,
    '-log(p)':Dlogps, 
    'Correlation (Pearson r)': Dcorrs,
    'Variable Category': DvarCat,
    'Variable Description': DvarDesc,
    'Variable ID': DvarID
    
})

In [74]:
dfT

,Contrast,-log(p),Correlation (Pearson r),Variable Category,Variable Description,Variable ID
0,rsfMRI-5,5.470336,0.177986,Diet,Drinking water intake (2.0),100150-2.0
1,T2 Nonlinear,5.735585,-0.137145,Lifestyle,Mother's age at death (0.0),3526-0.0
2,FA,5.798068,0.095623,Medical History,Diagnoses - ICD10 (E230 - E23.0 Hypopituitarism),41270-0.2428
3,FA,5.826818,0.095886,Medical History,Diagnoses - ICD10 (E274 - E27.4 Other and unsp...,41270-0.2456
4,FA,5.544933,0.093281,Medical History,Diagnoses - ICD10 (F009 - F00.9 Dementia in Al...,41270-0.2713
5,FA,5.840762,0.096013,Medical History,Diagnoses - ICD10 (M7969 - M79.69 Pain in limb...,41270-0.8949
6,FA,5.873064,0.096307,Medical History,Diagnoses - main ICD10 (E230 - E23.0 Hypopitui...,41202-0.2428
7,FA,5.814894,0.095777,Medical History,Diagnoses - secondary ICD10 (E230 - E23.0 Hypo...,41204-0.2428
8,FA,5.576710,0.093578,Medical History,Diagnoses - secondary ICD10 (F009 - F00.9 Deme...,41204-0.2713
9,FA,5.574542,0.093558,Medical History,Diagnoses - secondary ICD10 (G309 - G30.9 Alzh...,41204-0.3274


In [75]:
categories = sorted(list(set(dfT['Variable Category'].to_list())))

first_df = True

for cat in categories:
    df_tmp = dfT[(dfT['Variable Category']==cat)].sort_values('-log(p)', ascending=False)
#     df_tmp = df_tmp.head(10)
    if len(df_tmp) == 0:
        continue
    if first_df == True:
        df_print = df_tmp
        first_df = False
    else:
        df_print = pd.concat([df_print, df_tmp])
        


df_print = df_print.round(decimals=3)

df_print


,Contrast,-log(p),Correlation (Pearson r),Variable Category,Variable Description,Variable ID
0,rsfMRI-5,5.470,0.178,Diet,Drinking water intake (2.0),100150-2.0
1,T2 Nonlinear,5.736,-0.137,Lifestyle,Mother's age at death (0.0),3526-0.0
18,FA,6.196,0.099,Medical History,Treatment/medication code (1140884600 - metfor...,20003-0.1140884600
17,FA,6.080,0.098,Medical History,Treatment/medication code (1140872198 - sodium...,20003-0.1140872198
10,FA,5.878,0.096,Medical History,Diagnoses - secondary ICD10 (G403 - G40.3 Gene...,41204-0.3303
6,FA,5.873,0.096,Medical History,Diagnoses - main ICD10 (E230 - E23.0 Hypopitui...,41202-0.2428
15,FA,5.866,0.096,Medical History,Operative procedures - secondary OPCS4 (Z584 -...,41210-0.10584
14,FA,5.862,0.096,Medical History,Operative procedures - secondary OPCS4 (A843 -...,41210-0.533
12,FA,5.847,0.096,Medical History,Operative procedures - main OPCS4 (T811 - T81....,41200-0.7301
11,FA,5.841,0.096,Medical History,Operative procedures - OPCS4 (T811 - T81.1 Per...,41272-0.7301


In [76]:

df_print.to_csv('nIDPs_captured_by_SWIN.csv', index=False)

In [7]:
vars_df_converged

,modality,bonf,fdr,dataframe
0,T1_nonlinear_swin_c,5.442009,3.562704,idx ...
1,T2_nonlinear_swin_c,5.442009,3.632591,idx ...
2,tbss_FA_swin_c,5.442009,3.324640,idx ...
3,tbss_MD_swin_c,5.442009,3.537301,idx ...
4,rsfmri_0_swin_c,5.442009,3.681922,idx ...
5,swi_swin_c,5.442009,3.651473,idx ...
6,tfmri_swin_1_c,5.442009,3.533512,idx ...
7,tracts_swin_c,5.442009,3.474175,idx ...
8,tbss_MO_swin_c,5.442009,3.482356,idx ...
9,tbss_ISOVF_swin_s_c,5.442009,3.499444,idx ...


In [8]:
np.power(10, 5.783983)

608111.196940615

In [16]:
vars_df = vars_df.sort_index()

In [20]:
vars_df_converged = vars_df_converged.sort_index()

In [21]:
vars_df_converged

,modality,bonf,fdr,dataframe
0,T1_nonlinear_swin_c,5.442009,3.562704,idx ...
1,T2_nonlinear_swin_c,5.442009,3.632591,idx ...
2,tbss_FA_swin_c,5.442009,3.324640,idx ...
3,tbss_MD_swin_c,5.442009,3.537301,idx ...
4,rsfmri_0_swin_c,5.442009,3.681922,idx ...
5,swi_swin_c,5.442009,3.651473,idx ...
6,tfmri_swin_1_c,5.442009,3.533512,idx ...
7,tracts_swin_c,5.442009,3.474175,idx ...
8,tbss_MO_swin_c,5.442009,3.482356,idx ...
9,tbss_ISOVF_swin_s_c,5.442009,3.499444,idx ...


In [22]:
vars_df

,modality,bonf,fdr,dataframe
0,T1_nonlinear,5.442009,3.722608,idx ...
1,T2_nonlinear,5.442009,3.643544,idx ...
2,tbss_FA,5.442009,3.258847,idx ...
3,tbss_MD,5.442009,3.452753,idx ...
4,rsfmri_0,5.442009,3.389271,idx ...
5,swi,5.442009,3.490886,idx ...
6,tfmri_1,5.442009,3.608196,idx ...
7,tracts,5.442009,3.403591,idx ...
8,tbss_MO,5.442009,3.494445,idx ...
9,tbss_ISOVF_s,5.442009,3.308815,idx ...


In [23]:
for idx in range(len(vars_df)):

    modality = vars_df.iloc[idx].modality
    modalityEN = vars_df_converged.iloc[idx].modality

    
    print(modality, ' VS. ', modalityEN)
    print('----------------------------\n')

    df = vars_df.iloc[idx].dataframe
    bonf = vars_df.iloc[idx].bonf
    df = df[df.log_p_values >= bonf]
    
    
    dfc = vars_df_converged.iloc[idx].dataframe
    bonfC = vars_df_converged.iloc[idx].bonf
    dfc = dfc[dfc.log_p_values >= bonfC]

    print('nIDPs in {}: {}'.format(modality, len(df)))
    print('nIDPs in {}: {}'.format(modalityEN, len(dfc)))
    print('')
    print('Intersection = {}'.format(len(pd.merge(df, dfc, how='inner', on='idx'))))
    print('')
    dfm1 = df[df.idx.isin(list(set(df.idx.to_list()) - set(dfc.idx.to_list())))]
    dfm2 = dfc[dfc.idx.isin(list(set(dfc.idx.to_list()) - set(df.idx.to_list())))]
    print('{}-{} : {}'.format(modality, modalityEN, len(dfm1)))
#     print(dfm1[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])
    print('')
    print('{}-{} : {}'.format(modalityEN, modality, len(dfm2)))
#     print(dfm2[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])
    
    print('========================================================\n')

T1_nonlinear  VS.  T1_nonlinear_swin_c
----------------------------

nIDPs in T1_nonlinear: 42
nIDPs in T1_nonlinear_swin_c: 40

Intersection = 36

T1_nonlinear-T1_nonlinear_swin_c : 6

T1_nonlinear_swin_c-T1_nonlinear : 4

T2_nonlinear  VS.  T2_nonlinear_swin_c
----------------------------

nIDPs in T2_nonlinear: 25
nIDPs in T2_nonlinear_swin_c: 32

Intersection = 22

T2_nonlinear-T2_nonlinear_swin_c : 3

T2_nonlinear_swin_c-T2_nonlinear : 10

tbss_FA  VS.  tbss_FA_swin_c
----------------------------

nIDPs in tbss_FA: 62
nIDPs in tbss_FA_swin_c: 58

Intersection = 36

tbss_FA-tbss_FA_swin_c : 26

tbss_FA_swin_c-tbss_FA : 22

tbss_MD  VS.  tbss_MD_swin_c
----------------------------

nIDPs in tbss_MD: 53
nIDPs in tbss_MD_swin_c: 46

Intersection = 45

tbss_MD-tbss_MD_swin_c : 8

tbss_MD_swin_c-tbss_MD : 1

rsfmri_0  VS.  rsfmri_0_swin_c
----------------------------

nIDPs in rsfmri_0: 55
nIDPs in rsfmri_0_swin_c: 50

Intersection = 50

rsfmri_0-rsfmri_0_swin_c : 5

rsfmri_0_swin_c-rsf

In [24]:
idx1 = 5
idx2 = 6

modality = vars_df.iloc[idx1].modality
modalityEN = vars_df_converged.iloc[idx2].modality


print(modality, ' VS. ', modalityEN)
print('----------------------------\n')

df = vars_df.iloc[idx1].dataframe
bonf = vars_df.iloc[idx1].bonf
df = df[df.log_p_values >= bonf]


dfc = vars_df_converged.iloc[idx2].dataframe
bonfC = vars_df_converged.iloc[idx2].bonf
dfc = dfc[dfc.log_p_values >= bonfC]

print('nIDPs in {}: {}'.format(modality, len(df)))
print('nIDPs in {}: {}'.format(modalityEN, len(dfc)))
print('')
print('Intersection = {}'.format(len(pd.merge(df, dfc, how='inner', on='idx'))))
print('')
dfm1 = df[df.idx.isin(list(set(df.idx.to_list()) - set(dfc.idx.to_list())))]
dfm2 = dfc[dfc.idx.isin(list(set(dfc.idx.to_list()) - set(df.idx.to_list())))]
print('{}-{} : {}'.format(modality, modalityEN, len(dfm1)))
#     print(dfm1[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])
print('')
print('{}-{} : {}'.format(modalityEN, modality, len(dfm2)))
#     print(dfm2[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])

print('========================================================\n')

swi  VS.  tfmri_swin_1_c
----------------------------

nIDPs in swi: 46
nIDPs in tfmri_swin_1_c: 48

Intersection = 45

swi-tfmri_swin_1_c : 1

tfmri_swin_1_c-swi : 3



In [25]:
idx1 = 6
idx2 = 5

modality = vars_df.iloc[idx1].modality
modalityEN = vars_df_converged.iloc[idx2].modality


print(modality, ' VS. ', modalityEN)
print('----------------------------\n')

df = vars_df.iloc[idx1].dataframe
bonf = vars_df.iloc[idx1].bonf
df = df[df.log_p_values >= bonf]


dfc = vars_df_converged.iloc[idx2].dataframe
bonfC = vars_df_converged.iloc[idx2].bonf
dfc = dfc[dfc.log_p_values >= bonfC]

print('nIDPs in {}: {}'.format(modality, len(df)))
print('nIDPs in {}: {}'.format(modalityEN, len(dfc)))
print('')
print('Intersection = {}'.format(len(pd.merge(df, dfc, how='inner', on='idx'))))
print('')
dfm1 = df[df.idx.isin(list(set(df.idx.to_list()) - set(dfc.idx.to_list())))]
dfm2 = dfc[dfc.idx.isin(list(set(dfc.idx.to_list()) - set(df.idx.to_list())))]
print('{}-{} : {}'.format(modality, modalityEN, len(dfm1)))
#     print(dfm1[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])
print('')
print('{}-{} : {}'.format(modalityEN, modality, len(dfm2)))
#     print(dfm2[['idx', 'names', 'Categories', 'pearson_r', 'log_p_values']])

print('========================================================\n')

tfmri_1  VS.  swi_swin_c
----------------------------

nIDPs in tfmri_1: 48
nIDPs in swi_swin_c: 38

Intersection = 38

tfmri_1-swi_swin_c : 10

swi_swin_c-tfmri_1 : 0

